In [1]:
from optparse import OptionParser
import os
# from tqdm import tqdm

import numpy as np
from scipy.stats import beta, zscore
import tensorflow as tf

from basenji.dna_io import dna_1hot

np.random.seed(39)
import random

2024-10-11 15:38:05.645051: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-11 15:38:05.671219: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-11 15:38:05.671247: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-11 15:38:05.671979: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-11 15:38:05.677384: I tensorflow/core/platform/cpu_feature_guar

In [2]:
out_dir = "./tfrecords/"
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

In [3]:
seq_length = 32768 #2^15
bin_size = 512
seq_bins = seq_length // bin_size
split_labels = ["train", "valid", "test"]
diagonal_offset = 2

triu_tup = np.triu_indices(seq_bins, diagonal_offset)

In [4]:
### define CTCF motif 
ctcf_consensus = ['C','C','G','C','G','A','G','G','T','G','G','C','A','G']
ctcf_revcomp   = ['C','T','G','C','C','A','C','C','T','C','G','C','G','G']
ctcf_consensus= np.array(ctcf_consensus)
ctcf_revcomp = np.array(ctcf_revcomp)
motif_len = len(ctcf_consensus)
spacerA_len = 10

In [5]:
num_cell_types = 2

In [6]:
### define motif A
motifA_consensus = ['A','C','A','G','G','A']
motifA_revcomp   = ['T','C','C','T','G','T']
motifA_consensus= np.array(motifA_consensus)
motifA_revcomp = np.array(motifA_revcomp)
motifA_len = len(motifA_consensus)
spacerB_len = 6


In [7]:
### define motif B
motifB_consensus = ['T','T','G','A','C','T']
motifB_revcomp   = ['A','G','T','C','A','A']
motifB_consensus= np.array(motifB_consensus)
motifB_revcomp = np.array(motifB_revcomp)
motifB_len = len(motifA_consensus)
spacer_len = 6

In [8]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_map(matrix, vmin=-0.6, vmax=0.6, palette="RdBu_r", width=5, height=5):
    fig, axes = plt.subplots(1, 1, figsize=(width, height))

    sns.heatmap(
        matrix,
        vmin=vmin,
        vmax=vmax,
        cbar=False,
        cmap=palette,
        square=True,
        xticklabels=False,
        yticklabels=False,
        ax=axes
    )

    plt.tight_layout()
    plt.show()

In [9]:
def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [10]:
import numpy as np
from cooltools.lib.numutils import set_diag

def from_upper_triu(vector_repr, matrix_len, num_diags):
    z = np.zeros((matrix_len,matrix_len))
    triu_tup = np.triu_indices(matrix_len,num_diags)
    z[triu_tup] = vector_repr
    for i in range(-num_diags+1,num_diags):
        set_diag(z, np.nan, i)
    return z + z.T

/home1/smaruj/miniconda3/envs/basenji_py3.9_tf2.15/lib/python3.9/site-packages/cooltools/lib/numutils.py:652: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def iterative_correction_symmetric(
/home1/smaruj/miniconda3/envs/basenji_py3.9_tf2.15/lib/python3.9/site-packages/cooltools/lib/numutils.py:727: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def iterative_co

In [12]:
for split_label in split_labels:
    seqs_per_tfr = 8 #batch size
    if split_label == 'train': 
        num_seqs = 8 * 50
    if split_label == 'valid': 
        num_seqs = 8 * 10     
    if split_label == 'test': 
        num_seqs = 8 * 10      
    num_tfr = num_seqs // seqs_per_tfr

    # define options
    tf_opts = tf.io.TFRecordOptions(compression_type='ZLIB')

    # for ti in range(1):
    for ti in range(num_tfr):
        tfr_file = '%s/%s-%d.tfr' % (out_dir, split_label, ti)

        with tf.io.TFRecordWriter(tfr_file, tf_opts) as writer:

            for seq_idx in range(seqs_per_tfr):
                
                random_number = random.random()
                num_boundaries = np.random.randint(4,8)
                # print("initial num boundaries: ", num_boundaries)
                
                targetMatrix = np.zeros((seq_bins,seq_bins))
                    
                if random_number <= 0.33:
                    # print("no motif")
                    CTCFs_positions = np.sort(np.random.choice(np.arange(
                                    motif_len + spacer_len//2+1, seq_length - motif_len - spacer_len//2), num_boundaries, replace=False))
                    # all the maps look the same
                    boundary_positions = np.array([0] + list(CTCFs_positions) + [seq_length])
                    
                    for i in range(len(boundary_positions)-1):
                        s = boundary_positions[i] // bin_size
                        e = boundary_positions[i+1] // bin_size
                        targetMatrix[s:e,s:e] = 1
                        if s + 4 < seq_bins and e - 4 > 0:
                            targetMatrix[(s+2):(e-2),(s+2):(e-2)] = 0.75
                            targetMatrix[(s+4):(e-4),(s+4):(e-4)] = 0.5
                    
                    seq_dna = np.random.choice(['A','C','G','T'], size=seq_length, p= [.25,.25,.25,.25])
                    for i in range(1,len(boundary_positions)-1):
                        seq_dna[boundary_positions[i]-motif_len - spacer_len//2:boundary_positions[i]- spacer_len//2 ]  = ctcf_consensus
                        seq_dna[boundary_positions[i] + spacer_len//2: boundary_positions[i] + motif_len + spacer_len//2 ]  =ctcf_revcomp
                    
                    # collapse list
                    seq_dna = ''.join(seq_dna)

                    # 1 hot code
                    seq_1hot = dna_1hot(seq_dna)
                    
                    # compute targets
                    seq_targets = targetMatrix.astype('float16')
                    seq_targets = seq_targets[triu_tup].reshape((-1, 1))
                    
                    # print(seq_targets.shape)
                    
                    full_seq_targets = np.array([seq_targets] + [seq_targets])
                    full_seq_targets = np.transpose(full_seq_targets, (1, 0, 2))
                    
                    # print(full_seq_targets.shape)
                    # test = from_upper_triu(full_seq_targets[1,:,:].flatten(), matrix_len=64, num_diags=2)
                    # print(test.shape)
                    # plot_map(test)
                    
                    # same target for all the cell types
                    # make example
                    example = tf.train.Example(features=tf.train.Features(feature={
                    'sequence': _bytes_feature(seq_1hot.flatten().tostring()),
                    'target': _bytes_feature(full_seq_targets[:, :].flatten().tostring()),
                    }))

                    # write example
                    writer.write(example.SerializeToString())
                    
                elif random_number > 0.33 and random_number <= 0.66:
                    # print("motif A")
                    num_boundaries += 1
                    # print("num boundaries: ", num_boundaries)
                    positions = np.random.choice(np.arange(
                                    motif_len + spacer_len//2+1, seq_length - motif_len - spacer_len//2), num_boundaries, replace=False)
                    CTCFs_positions = np.sort(positions[:-1])
                    CTCF_boundary_positions = np.array([0] + list(CTCFs_positions) + [seq_length])
                    
                    motifA_position = positions[-1]
                    CTCF_A_boundary_positions = np.sort(np.array([0] + list(positions) + [seq_length]))
                    
                    # type B
                    targetMatrixB = targetMatrix.copy()
                    for i in range(len(CTCF_boundary_positions)-1):
                        s = CTCF_boundary_positions[i] // bin_size
                        e = CTCF_boundary_positions[i+1] // bin_size
                        targetMatrixB[s:e,s:e] = 1
                        if s + 4 < seq_bins and e - 4 > 0:
                            targetMatrixB[(s+2):(e-2),(s+2):(e-2)] = 0.75
                            targetMatrixB[(s+4):(e-4),(s+4):(e-4)] = 0.5
                    
                    # print("typeB")
                    # plot_map(targetMatrixB)
                    
                    # type A
                    targetMatrixA = targetMatrix.copy()
                    for i in range(len(CTCF_A_boundary_positions)-1):
                        s = CTCF_A_boundary_positions[i] // bin_size
                        e = CTCF_A_boundary_positions[i+1] // bin_size
                        targetMatrixA[s:e,s:e] = 1
                        if s + 4 < seq_bins and e - 4 > 0:
                            targetMatrixA[(s+2):(e-2),(s+2):(e-2)] = 0.75
                            targetMatrixA[(s+4):(e-4),(s+4):(e-4)] = 0.5
                    
                    # print("typeA")
                    # plot_map(targetMatrixA)
                    
                    seq_dna = np.random.choice(['A','C','G','T'], size=seq_length, p= [.25,.25,.25,.25])
                    for i in range(1,len(CTCF_boundary_positions)-1):
                        seq_dna[CTCF_boundary_positions[i]-motif_len - spacer_len//2:CTCF_boundary_positions[i]- spacer_len//2 ]  = ctcf_consensus
                        seq_dna[CTCF_boundary_positions[i] + spacer_len//2: CTCF_boundary_positions[i] + motif_len + spacer_len//2 ]  =ctcf_revcomp
                    # adding motif A
                    seq_dna[motifA_position - motifA_len - spacerA_len//2:motifA_position - spacerA_len//2 ]  = motifA_consensus
                    seq_dna[motifA_position + spacerA_len//2:motifA_position + motifA_len + spacerA_len//2 ]  = motifA_revcomp
                    
                    # collapse list
                    seq_dna = ''.join(seq_dna)

                    # 1 hot code
                    seq_1hot = dna_1hot(seq_dna)
                    
                    # compute targets
                    seq_targets_A = targetMatrixA.astype('float16')
                    seq_targets_A = seq_targets_A[triu_tup].reshape((-1, 1))
                    
                    seq_targets_B = targetMatrixB.astype('float16')
                    seq_targets_B = seq_targets_B[triu_tup].reshape((-1, 1))
                    
                    full_seq_targets = np.array([seq_targets_A] + [seq_targets_B])
                    full_seq_targets = np.transpose(full_seq_targets, (1, 0, 2))
                    # np.concatenate((vector1, vector2), axis=0)
                    
                    # for cell type A there is one more boundary at the location of motif A
                    # make example
                    example = tf.train.Example(features=tf.train.Features(feature={
                    'sequence': _bytes_feature(seq_1hot.flatten().tostring()),
                    'target': _bytes_feature(full_seq_targets[:, :].flatten().tostring())
                    }))
                    # write example
                    writer.write(example.SerializeToString())
                    
                    # print(seq_1hot.flatten().shape)
                    # print(full_seq_targets[:, :].flatten().shape)
                    
                else:
                    # print("motif B")
                    all_positions = np.random.choice(np.arange(
                                    motif_len + spacer_len//2+1, seq_length - motif_len - spacer_len//2), num_boundaries, replace=False)
                    motifB_position = all_positions[-1]
                    positions = all_positions[:-1]
                    boundary_positions = np.sort(np.array([0] + list(positions) + [seq_length]))
                    boundary_positions_typeA = np.sort(np.array([0] + [motifB_position] + list(positions) + [seq_length]))
                    
                    # type A
                    targetMatrixA = targetMatrix.copy()
                    for i in range(len(boundary_positions_typeA)-1):
                        s = boundary_positions_typeA[i] // bin_size
                        e = boundary_positions_typeA[i+1] // bin_size
                        targetMatrixA[s:e,s:e] = 1
                        if s + 4 < seq_bins and e - 4 > 0:
                            targetMatrixA[(s+2):(e-2),(s+2):(e-2)] = 0.75
                            targetMatrixA[(s+4):(e-4),(s+4):(e-4)] = 0.5
                    
                    # print("typeA")
                    # plot_map(targetMatrixA)
                    
                    # type B
                    targetMatrixB = targetMatrix.copy()
                    for i in range(len(boundary_positions)-1):
                        s = boundary_positions[i] // bin_size
                        e = boundary_positions[i+1] // bin_size
                        targetMatrixB[s:e,s:e] = 1
                        if s + 4 < seq_bins and e - 4 > 0:
                            targetMatrixB[(s+2):(e-2),(s+2):(e-2)] = 0.75
                            targetMatrixB[(s+4):(e-4),(s+4):(e-4)] = 0.5
                    
                    # print("typeB")
                    # plot_map(targetMatrixB)
                    
                    seq_dna = np.random.choice(['A','C','G','T'], size=seq_length, p= [.25,.25,.25,.25])
                    for i in range(1,len(boundary_positions)-1):
                        seq_dna[boundary_positions[i]-motif_len - spacer_len//2:boundary_positions[i]- spacer_len//2 ]  = ctcf_consensus
                        seq_dna[boundary_positions[i] + spacer_len//2: boundary_positions[i] + motif_len + spacer_len//2 ]  =ctcf_revcomp
                    # adding motif A
                    seq_dna[motifB_position - motifB_len - spacerB_len//2:motifB_position - spacerB_len//2 ]  = motifB_consensus
                    seq_dna[motifB_position + spacerB_len//2:motifB_position + motifB_len + spacerB_len//2 ]  = motifB_revcomp
                    
                    # collapse list
                    seq_dna = ''.join(seq_dna)

                    # 1 hot code
                    seq_1hot = dna_1hot(seq_dna)
                    
                    # compute targets
                    seq_targets_A = targetMatrixA.astype('float16')
                    seq_targets_A = seq_targets_A[triu_tup].reshape((-1, 1))
                    
                    seq_targets_B = targetMatrixB.astype('float16')
                    seq_targets_B = seq_targets_B[triu_tup].reshape((-1, 1))
                    
                    full_seq_targets = np.array([seq_targets_A] + [seq_targets_B])
                    full_seq_targets = np.transpose(full_seq_targets, (1, 0, 2))
                    
                    # for cell type B there is one less boundary at the location of motif B (merged)
                    # make example
                    example = tf.train.Example(features=tf.train.Features(feature={
                    'sequence': _bytes_feature(seq_1hot.flatten().tostring()),
                    'target': _bytes_feature(full_seq_targets[:, :].flatten().tostring())
                    }))
                    # write example
                    writer.write(example.SerializeToString())

/tmp/SLURM_23930713/ipykernel_36831/1943612459.py:215: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'sequence': _bytes_feature(seq_1hot.flatten().tostring()),
/tmp/SLURM_23930713/ipykernel_36831/1943612459.py:216: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'target': _bytes_feature(full_seq_targets[:, :].flatten().tostring())
/tmp/SLURM_23930713/ipykernel_36831/1943612459.py:144: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'sequence': _bytes_feature(seq_1hot.flatten().tostring()),
/tmp/SLURM_23930713/ipykernel_36831/1943612459.py:145: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'target': _bytes_feature(full_seq_targets[:, :].flatten().tostring())
/tmp/SLURM_23930713/ipykernel_36831/1943612459.py:71: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'sequence': _bytes_feature(seq_1hot.flatten().tostring()),
/tmp/SLURM_23930713/ipykernel_36831/1943612459.py:72: 